# Homework 1 Problem #2

In [ ]:
import multiprocessing as mp
import numpy as np
%matplotlib inline
import matplotlib.pyplot as mplt
import pandas as pd
from IPython.display import display, HTML

# Minimum Function

In [ ]:
N = 10**7
x = np.random.rand(N)

def inmin(y,q):
    q.put(min(y))

def parmin(x,nc):
    # Divide the array into chunks
    N = len(x)
    ls = N//nc + 1
    # ys is list of chunks
    ys = [x[ls*i:ls*i+ls] for i in range(nc)]
    # make a return queue
    q = mp.Queue()
    # We want to send chunks to seperate processes
    jobs = [mp.Process(target=inmin,args = (y,q)) for y in ys]
    for job in jobs:
        job.start()
    # Then wait for processes to send mins back
    for job in jobs:
        job.join()
    # min mins
    mins = []
    while not q.empty():
        mins.append(q.get())
    tmin = min(mins)
    # Return total min
    return tmin

parmin(x,4) - min(x)

# Maximum Function

In [ ]:
N = 10**6
x = np.random.rand(N)

def inmax(y,q):
    q.put(max(y))

def parmax(x,nc):
    # Divide the array into chunks
    N = len(x)
    ls = N//nc + 1
    # ys is list of chunks
    ys = [x[ls*i:ls*i+ls] for i in range(nc)]
    # make a return queue
    q = mp.Queue()
    # We want to send chunks to seperate processes
    jobs = [mp.Process(target=inmax,args = (y,q)) for y in ys]
    for job in jobs:
        job.start()
    # Then wait for processes to send maxs back
    for job in jobs:
        job.join()
    # max maxs
    maxs = []
    while not q.empty():
        maxs.append(q.get())
    tmax = max(maxs)
    # Return total max
    return tmax

parmax(x,4) - max(x)

# Mean Function

In [ ]:
N = 10**6
x = np.random.rand(N)

def insum(y,q):
    q.put(sum(y))

def parmean(x,nc):
    # Divide the array into chunks
    N = len(x)
    ls = N//nc + 1
    # ys is list of chunks
    ys = [x[ls*i:ls*i+ls] for i in range(nc)]
    # make a return queue
    q = mp.Queue()
    # We want to send chunks to seperate processes
    jobs = [mp.Process(target=insum,args = (y,q)) for y in ys]
    for job in jobs:
        job.start()
    # Then wait for processes to send sums back
    for job in jobs:
        job.join()
    # Sum sums
    sums = 0
    while not q.empty():
        sums += q.get()
    # Divide sum by length(N)
    xbar = sums/N
    # Return mean
    return xbar

parmean(x,10) - np.mean(x)

# Standard Deviation Function

In [ ]:
N = 10**4
x = np.random.rand(N)

def instd(y,q,xbar):
    q.put(sum([(xval - xbar)**2 for xval in y]))

def parstd(x,nc):
    # Get the mean of x
    xbar = parmean(x,nc)
    
    # Divide the array into chunks
    N = len(x)
    ls = N//nc + 1
    # ys is list of chunks
    ys = [x[ls*i:ls*i+ls] for i in range(nc)]
    # make a return queue
    q = mp.Queue()
    # We want to send chunks to seperate processes
    jobs = [mp.Process(target=instd,args = (y,q,xbar)) for y in ys]
    for job in jobs:
        job.start()
    # Then wait for processes to send sums squared back
    for job in jobs:
        job.join()
    # Sum sums
    sums = 0
    while not q.empty():
        sums += q.get()
    # Divide sum by length(N)-1
    std = np.sqrt(sums/(N))
    # Return standard deviation
    return std

print(parstd(x,10) - np.std(x))

# Timings Section

In [ ]:
N = 10**6
x = np.random.rand(N)
nc = 4
min_timings = []
max_timings = []
std_timings = []

In [ ]:
for i in range(nc):
    times = %%timeit -o -n 1 pass
    parmin(x,nc)
    min_timings.append(times.average)

In [ ]:
for i in range(nc):
    times = %%timeit -o -n 1 pass
    parmax(x,nc)
    max_timings.append(times.average)

In [ ]:
for i in range(nc):
    times = %%timeit -o -n 1 pass
    parstd(x,nc)
    std_timings.append(times.average)

In [ ]:
ncores = [i+1 for i in range(nc)]
t_ep = [(i+1)**0 for i in range(nc)]

In [ ]:
sp_min  = [min_timings[0]/min_timings[i] for i in range(len(min_timings))]
sp_max  = [max_timings[0]/max_timings[i] for i in range(len(max_timings))]
sp_std  = [std_timings[0]/std_timings[i] for i in range(len(std_timings))]

In [ ]:
ep_min = [sp_min[i]/nc for i in range(nc)]
ep_max = [sp_max[i]/nc for i in range(nc)]
ep_std = [sp_std[i]/nc for i in range(nc)]

In [ ]:
d_min = [[i+1 for i in range(nc)],min_timings,sp_min,ncores,ep_min,t_ep]
d_max = [[i+1 for i in range(nc)],max_timings,sp_max,ncores,ep_max,t_ep]
d_std = [[i+1 for i in range(nc)],std_timings,sp_std,ncores,ep_std,t_ep]

In [ ]:
columns = ["Number of Processors","Time","Speed-Up","Theoretical Speed-Up","Efficiency","Theoretical Efficiency"]
timings_min = pd.DataFrame(data=d_min,index=columns).T
timings_max = pd.DataFrame(data=d_max,index=columns).T
timings_std = pd.DataFrame(data=d_std,index=columns).T

In [ ]:
display(HTML(timings_min.to_html(index=False)))

In [ ]:
min_plot = timings_min.plot(x = "Number of Processors",y = ["Speed-Up","Theoretical Speed-Up"],style="*-",loglog=True)
min_plot.set_title("Minimum Function",fontsize = 18)
min_plot.set_xlabel("Processors")
min_plot.set_ylabel("Speed-Up")

In [ ]:
min_plot = timings_min.plot(x = "Number of Processors",y = ["Efficiency","Theoretical Efficiency"],style="*-",logx=True)
min_plot.set_title("Minimum Function",fontsize = 18)
min_plot.set_xlabel("Processors")
min_plot.set_ylabel("Efficiency")

In [ ]:
display(HTML(timings_max.to_html(index=False)))

In [ ]:
max_plot = timings_max.plot(x = "Number of Processors",y = ["Speed-Up","Theoretical Speed-Up"],style="*-",loglog=True)
max_plot.set_title("Maximum Function",fontsize = 18)
max_plot.set_xlabel("Processors")
max_plot.set_ylabel("Speed-Up")

In [ ]:
max_plot = timings_max.plot(x = "Number of Processors",y = ["Efficiency","Theoretical Efficiency"],style="*-",logx=True)
max_plot.set_title("Maximum Function",fontsize = 18)
max_plot.set_xlabel("Processors")
max_plot.set_ylabel("Efficiency")

In [ ]:
display(HTML(timings_std.to_html(index=False)))

In [ ]:
std_plot = timings_std.plot(x = "Number of Processors",y = ["Speed-Up","Theoretical Speed-Up"],style="*-",loglog=True)
std_plot.set_title("Standard Deviation Function",fontsize = 18)
std_plot.set_xlabel("Processors")
std_plot.set_ylabel("Speed-Up")

In [ ]:
std_plot = timings_std.plot(x = "Number of Processors",y = ["Efficiency","Theoretical Efficiency"],style="*-",logx=True)
std_plot.set_title("Standard Deviation",fontsize = 18)
std_plot.set_xlabel("Processors")
std_plot.set_ylabel("Efficiency")